# Tutorial T03e: Reproducing Bhat (2018) Table 1 — MVNCD Accuracy Evaluation

This tutorial reproduces the systematic Monte Carlo evaluation from Table 1 of:

> Bhat, C. R. (2018). New Matrix-Based Methods for the Analytic Evaluation
> of the MVNCD Function. *Transportation Research Part B*, 109: 238-256.

Table 1 evaluates MVNCD approximation accuracy across dimensions H=5,7,10,12,
15,18,20 using 1000 random correlation matrices per H value. For each, it
reports MAE, MAPE, %MAE>0.005, %MAPE>2%, and computation time.

**What you will learn:**
- How the paper's Monte Carlo test design works (Section 3.1)
- How random correlation matrices are generated (low vs high correlation)
- How to benchmark MVNCD methods systematically
- How our Python implementation compares to the paper's GAUSS results

**Prerequisites:** t03a (MVNCD methods overview).

**Configuration:**
- `N_TESTS = 100` by default for fast execution (~1 min per H)
- Set `N_TESTS = 1000` for full replication of the paper (~10 min per H)
- `H_VALUES = [5, 7, 10]` by default; add `[12, 15, 18, 20]` for full table

In [ ]:
import os, sys, time
import numpy as np
from scipy.stats import multivariate_normal as scipy_mvn

np.set_printoptions(precision=4, suppress=True)
import pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent / "src"))

from pybhatlib.gradmvn import mvncd

## Configuration

In [ ]:
# Number of random test cases per H value.
# Paper uses 1000; we default to 100 for fast execution.
N_TESTS = 100

# Dimensions to test.
# Paper uses [5, 7, 10, 12, 15, 18, 20]; we default to small ones.
H_VALUES = [5, 7, 10]

# Methods to benchmark (analytic + simulation).
METHODS = ["me", "ovus", "ovbs", "bme", "tvbs"]
SSJ_CONFIGS = [(500, "ssj-500"), (10000, "ssj-10k")]

# Random seed for reproducibility.
SEED = 42

# Dimension threshold: use scipy for H <= this, SSJ for H > this.
SCIPY_THRESHOLD = 10

# SSJ reference draws for H > SCIPY_THRESHOLD.
SSJ_REF_DRAWS = 50000

## Step 1: Random Correlation Matrix Generation (Section 3.1)

The paper generates 1000 random correlation matrices per dimension H,
split equally into two groups:

**Low correlation (50%):**
```
C = R @ R.T + 10 * diag(r_u)
```
where R is H×H uniform[0,1], r_u is H uniform[0,1].
The large diagonal boost (10×) weakens off-diagonal correlations.

**High correlation (50%):**
```
C = R @ R.T + 0 * diag(r_u)   (no diagonal boost)
```
Off-diagonal correlations remain strong.

Both are normalized to correlation matrices: C_ij / sqrt(C_ii × C_jj).

**Upper integration limits** a (per test):
- Half from U[0, √H] — all positive limits
- Half from U[-√H/2, √H] — mixed positive/negative limits

In [ ]:
def generate_test_cases(H, n_tests, rng):
    """Generate random correlation matrices and integration limits.

    Follows Bhat (2018) Section 3.1, p.248.
    """
    n_low = n_tests // 2
    test_cases = []

    for i in range(n_tests):
        # --- Correlation matrix ---
        R = rng.uniform(0, 1, (H, H))
        C = R @ R.T

        if i < n_low:
            # Low correlation: add large diagonal boost
            r_u = rng.uniform(0, 1, H)
            C += 10.0 * np.diag(r_u)

        # Normalize to correlation matrix
        d = np.sqrt(np.diag(C))
        sigma = C / np.outer(d, d)
        sigma = (sigma + sigma.T) / 2.0
        np.fill_diagonal(sigma, 1.0)

        # --- Integration limits ---
        sqrtH = np.sqrt(H)
        if i % 2 == 0:
            a = rng.uniform(0, sqrtH, H)
        else:
            a = rng.uniform(-sqrtH / 2, sqrtH, H)

        test_cases.append((a, sigma))

    return test_cases

## Step 2: Reference CDF Values

In [ ]:
def compute_reference(a, sigma, H, rng_ref):
    """Compute reference CDF value.

    For H <= SCIPY_THRESHOLD: uses scipy.stats.multivariate_normal.cdf
    (Genz 1992 algorithm, high accuracy).

    For H > SCIPY_THRESHOLD: uses SSJ with many draws.
    """
    if H <= SCIPY_THRESHOLD:
        try:
            return float(scipy_mvn.cdf(a, mean=np.zeros(H), cov=sigma))
        except np.linalg.LinAlgError:
            return mvncd(a, sigma, method="ssj", n_draws=SSJ_REF_DRAWS,
                         seed=int(rng_ref.integers(1, 2**31)))
    else:
        return mvncd(a, sigma, method="ssj", n_draws=SSJ_REF_DRAWS,
                     seed=int(rng_ref.integers(1, 2**31)))

## Step 3: Paper Table 1 Values for Comparison

Bhat (2018) Table 1 (p.250) reports MAE, MAPE, %MAE>0.005, %MAPE>2% for each method and dimension.

In [ ]:
# Format: {H: {method: (MAE, MAPE, pct_mae_005, pct_mape_2)}}
PAPER_TABLE1 = {
    5:  {"me": (0.0025, 1.78, 11.5, 18.5),
         "ovus": (0.0019, 1.52, 9.2, 15.7),
         "bme": (0.0015, 1.32, 7.9, 13.7),
         "tvbs": (0.0008, 0.82, 3.2, 6.3),
         "ovbs": (0.0012, 0.98, 5.3, 9.1)},
    7:  {"me": (0.0048, 4.32, 22.8, 33.3),
         "ovus": (0.0025, 2.55, 13.0, 19.5),
         "bme": (0.0023, 2.33, 11.7, 18.7),
         "tvbs": (0.0010, 1.09, 4.2, 8.1),
         "ovbs": (0.0014, 1.49, 6.9, 11.3)},
    10: {"me": (0.0068, 8.12, 34.3, 47.2),
         "ovus": (0.0028, 3.78, 15.4, 23.2),
         "bme": (0.0026, 3.36, 14.4, 22.7),
         "tvbs": (0.0011, 1.52, 5.3, 10.2),
         "ovbs": (0.0015, 2.02, 7.6, 13.4)},
}

## Step 4: Run Benchmarks

For each H value, we generate N_TESTS random test cases, compute reference values,
then evaluate all methods and compare with the paper.

In [ ]:
all_results = {}
rng = np.random.default_rng(SEED)

for H in H_VALUES:
    print(f"\n{'=' * 72}")
    print(f"  H = {H}  (generating {N_TESTS} random test cases)")
    print(f"{'=' * 72}")

    # Generate test cases
    test_cases = generate_test_cases(H, N_TESTS, rng)

    # Compute reference values
    ref_method = "scipy" if H <= SCIPY_THRESHOLD else f"SSJ({SSJ_REF_DRAWS:,})"
    print(f"\n  Computing reference values via {ref_method}...", end="", flush=True)
    t0 = time.perf_counter()
    rng_ref = np.random.default_rng(SEED + 1000)
    refs = []
    for a, sigma in test_cases:
        refs.append(compute_reference(a, sigma, H, rng_ref))
    ref_time = time.perf_counter() - t0
    refs = np.array(refs)
    print(f" done ({ref_time:.1f}s)")

    # Skip cases where reference is essentially zero
    valid = refs > 1e-10
    n_valid = valid.sum()
    if n_valid < N_TESTS:
        print(f"  Note: {N_TESTS - n_valid} cases with ref ~ 0 excluded from MAPE")

    # --- Benchmark each method ---
    results = {}

    # Analytic methods
    for method in METHODS:
        t0 = time.perf_counter()
        pvals = np.zeros(N_TESTS)
        for i, (a, sigma) in enumerate(test_cases):
            pvals[i] = mvncd(a, sigma, method=method, seed=42)
        elapsed = time.perf_counter() - t0

        ae = np.abs(pvals - refs)
        ape = np.where(valid, ae / np.maximum(refs, 1e-15) * 100, 0.0)

        mae = ae.mean()
        mape = ape[valid].mean() if n_valid > 0 else 0.0
        pct_mae_005 = (ae > 0.005).mean() * 100
        pct_mape_2 = (ape[valid] > 2.0).mean() * 100 if n_valid > 0 else 0.0

        results[method] = (mae, mape, pct_mae_005, pct_mape_2, elapsed)

    # SSJ methods
    for n_draws, label in SSJ_CONFIGS:
        t0 = time.perf_counter()
        pvals = np.zeros(N_TESTS)
        for i, (a, sigma) in enumerate(test_cases):
            pvals[i] = mvncd(a, sigma, method="ssj", n_draws=n_draws, seed=42)
        elapsed = time.perf_counter() - t0

        ae = np.abs(pvals - refs)
        ape = np.where(valid, ae / np.maximum(refs, 1e-15) * 100, 0.0)

        mae = ae.mean()
        mape = ape[valid].mean() if n_valid > 0 else 0.0
        pct_mae_005 = (ae > 0.005).mean() * 100
        pct_mape_2 = (ape[valid] > 2.0).mean() * 100 if n_valid > 0 else 0.0

        results[label] = (mae, mape, pct_mae_005, pct_mape_2, elapsed)

    all_results[H] = results

    # --- Print results table ---
    print(f"\n  {'Method':>10s} {'MAE':>10s} {'MAPE%':>8s} {'%MAE>.005':>10s}"
          f" {'%MAPE>2':>8s} {'Time(s)':>8s}")
    print(f"  {'-' * 58}")
    for label in METHODS + [cfg[1] for cfg in SSJ_CONFIGS]:
        mae, mape, pct_mae, pct_mape, elapsed = results[label]
        print(f"  {label:>10s} {mae:>10.4f} {mape:>7.2f}% {pct_mae:>9.1f}%"
              f" {pct_mape:>7.1f}% {elapsed:>7.1f}s")

    # --- Compare with paper Table 1 ---
    if H in PAPER_TABLE1:
        print(f"\n  Comparison with Bhat (2018) Table 1 (H={H}):")
        print(f"  {'Method':>10s} {'Ours':>8s} {'Paper':>8s} {'Ours':>10s}"
              f" {'Paper':>8s}")
        print(f"  {'':>10s} {'MAPE%':>8s} {'MAPE%':>8s} {'MAE':>10s}"
              f" {'MAE':>8s}")
        print(f"  {'-' * 50}")
        for method in METHODS:
            if method in PAPER_TABLE1[H]:
                our_mae, our_mape = results[method][0], results[method][1]
                paper_mae, paper_mape = PAPER_TABLE1[H][method][:2]
                mape_better = "*" if our_mape < paper_mape else " "
                mae_better = "*" if our_mae < paper_mae else " "
                print(f"  {method:>10s} {our_mape:>7.2f}%{mape_better}"
                      f" {paper_mape:>7.2f}% {our_mae:>9.4f}{mae_better}"
                      f" {paper_mae:>7.4f}")
        print(f"\n  * = our implementation outperforms paper")

## Step 5: Summary Across All Dimensions

In [ ]:
print(f"\n{'=' * 72}")
print(f"  Summary: MAPE(%) across dimensions")
print(f"{'=' * 72}")

all_labels = METHODS + [cfg[1] for cfg in SSJ_CONFIGS]
header = f"  {'Method':>10s}" + "".join(f" {'H='+str(H):>8s}" for H in H_VALUES)
print(header)
print(f"  {'-' * (10 + 9 * len(H_VALUES))}")

for label in all_labels:
    row = f"  {label:>10s}"
    for H in H_VALUES:
        mape = all_results[H][label][1]
        row += f" {mape:>7.2f}%"
    print(row)

# Paper comparison
if any(H in PAPER_TABLE1 for H in H_VALUES):
    print(f"\n  Paper Table 1 MAPE(%) for reference:")
    for method in METHODS:
        row = f"  {method+'-paper':>10s}"
        for H in H_VALUES:
            if H in PAPER_TABLE1 and method in PAPER_TABLE1[H]:
                row += f" {PAPER_TABLE1[H][method][1]:>7.2f}%"
            else:
                row += f" {'n/a':>8s}"
        print(row)

## Interpretation

**1. Accuracy ranking** (consistent with paper):
TVBS > OVBS > BME > OVUS > ME. TVBS is the most accurate analytic method at all dimensions.

**2. Our implementation vs paper:**
Our MAPE values are generally *lower* (better) than the paper because we use scipy's exact
bivariate/trivariate/quadrivariate CDFs as building blocks, while the original GAUSS code
used numerical approximations for these base CDFs.

**3. Dimension scaling:**
All analytic methods degrade with increasing H, but TVBS degrades most gracefully.
ME degrades fastest because it only uses univariate conditioning (no screening correction).

**4. SSJ simulation:**
SSJ accuracy improves with `n_draws` but is slower. SSJ(10000) is competitive with the
best analytic methods. SSJ is recommended when H > 20 or when highest accuracy is needed.

**5. Reference values:**
- For H ≤ 10: `scipy.stats.multivariate_normal.cdf` (Genz algorithm)
- For H > 10: SSJ with 50,000 draws (scipy unreliable at high dimensions)

To reproduce the full paper Table 1, set `N_TESTS = 1000` and `H_VALUES = [5, 7, 10, 12, 15, 18, 20]`.

**Reference:**
Bhat, C. R. (2018). New Matrix-Based Methods for the Analytic Evaluation of the MVNCD Function.
*Transportation Research Part B*, 109: 238-256.

**Next:** See `t04a_mnp_basic` for MNP estimation using these MVNCD methods.